In [3]:
import itertools

def get_value(word, substitution):
    """Calculate the numeric value of a word based on letter-digit mapping."""
    s = 0
    factor = 1
    for letter in reversed(word):
        s += factor * substitution.get(letter, 0)
        factor *= 10
    return s

def is_valid_solution(solution, words, result):
    """Check if the current mapping solution is valid."""
    if any(solution.get(word[0], 0) == 0 for word in words + [result]): 
        return False
    word_values = [get_value(word, solution) for word in words]
    result_value = get_value(result, solution)
    return sum(word_values) == result_value

def dfs(letters, digits, words, result, solution, solutions):
    """Depth-First Search implementation for solving cryptarithmetic puzzles."""
    if len(solution) == len(letters):  
        if is_valid_solution(solution, words, result):
            solutions.append(solution.copy())
        return
    
    current_letter = letters[len(solution)]
    for digit in digits:
        if digit not in solution.values():  
            solution[current_letter] = digit
            dfs(letters, digits, words, result, solution, solutions)
            del solution[current_letter]  # Backtrack

def solve_cryptarithmetic_puzzle(equation):
    left, right = equation.lower().replace(' ', '').split('=')
    words = left.split('+')
    letters = set(right)
    for word in words:
        for letter in word:
            letters.add(letter)
    letters = list(letters)

    solutions = []
    dfs(letters, range(10), words, right, {}, solutions)
    
    # Print solutions
    for solution in solutions:
        print(' + '.join(str(get_value(word, solution)) for word in words) + " = {} (mapping: {})".format(get_value(right, solution), solution))

if __name__ == '__main__':
    solve_cryptarithmetic_puzzle('SEND + MORE = MONEY')


9567 + 1085 = 10652 (mapping: {'d': 7, 's': 9, 'r': 8, 'y': 2, 'o': 0, 'n': 6, 'e': 5, 'm': 1})


In [9]:
import itertools
import heapq

def get_value(word, substitution):
    """Calculate the numeric value of a word based on letter-digit mapping."""
    s = 0
    factor = 1
    for letter in reversed(word):
        s += factor * substitution.get(letter, 0)
        factor *= 10
    return s

def par_sum(words, solution):
    """Calculate the sum of words for the current partial solution."""
    return sum(get_value(word, solution) for word in words)

def target_val(result, solution):
    """Calculate the numeric value of the result for the current partial solution."""
    return get_value(result, solution)

def heuristic(words, result, solution):
    """Heuristic based on the difference between the partial sum and the target value."""
    partial_sum = par_sum(words, solution)
    target_value = target_val(result, solution)
    # Return absolute difference as the heuristic value
    return abs(partial_sum - target_value)

def is_valid_partial_solution(solution, words, result):
    """Check if the partial mapping does not violate constraints, including leading zeros."""
    if any(solution.get(word[0], None) == 0 for word in words + [result]):
        return False
    return True

def a_star_search(letters, digits, words, result):
    """A* Search implementation for solving cryptarithmetic puzzles."""
    queue = []  # Priority queue
    visited = set()  # Keep track of visited states to avoid repetition
   
    initial_state = tuple([None] * len(letters))  # Represent unassigned letters with None
    heapq.heappush(queue, (0, initial_state, 0))  # Initial push with heuristic, state, and zero assigned letters

    while queue:
        current_priority, state, assigned_letters = heapq.heappop(queue)
        
        # Convert state tuple back to solution dictionary for ease of use
        solution = {letters[i]: state[i] for i in range(assigned_letters) if state[i] is not None}
        
        if assigned_letters == len(letters):
            if par_sum(words, solution) == target_val(result, solution):
                return solution  # Solution found
            continue

        current_letter_index = assigned_letters
        for digit in digits:
            if digit not in solution.values():
                new_state = list(state[:assigned_letters]) + [digit] + list(state[assigned_letters + 1:])
                
                if tuple(new_state) in visited:
                    continue  # Skip already visited states
                
                visited.add(tuple(new_state))
                new_solution = solution.copy()
                new_solution[letters[current_letter_index]] = digit

                if not is_valid_partial_solution(new_solution, words, result):
                    continue  # Skip invalid partial solutions

                new_priority = heuristic(words, result, new_solution)
                heapq.heappush(queue, (new_priority, tuple(new_state), assigned_letters + 1))

    return None  # No solution found

def cryp_puzzle_A_star(equation):
    left, right = equation.lower().replace(' ', '').split('=')
    words = left.split('+')
    letters = set(right)
    for word in words:
        for letter in word:
            letters.add(letter)
    letters = list(letters)

    solution = a_star_search(letters, range(10), words, right)
    
    if solution:
        solution_str = ' + '.join(str(get_value(word, solution)) for word in words) + " = {} (mapping: {})".format(get_value(right, solution), {k: v for k, v in sorted(solution.items())})
        print(solution_str)
    else:
        print("No solution found.")

if __name__ == '__main__':
    cryp_puzzle_A_star('SEND + MORE = MONEY')


9567 + 1085 = 10652 (mapping: {'d': 7, 'e': 5, 'm': 1, 'n': 6, 'o': 0, 'r': 8, 's': 9, 'y': 2})
